In [3]:
pip install selenium

Note: you may need to restart the kernel to use updated packages.


In [4]:
pip install webdriver_manager selenium requests beautifulsoup4 jupyter

Note: you may need to restart the kernel to use updated packages.


In [6]:
pip install --upgrade webdriver-manager

Note: you may need to restart the kernel to use updated packages.


In [7]:
from selenium import webdriver 
from selenium.webdriver.common.keys import Keys 
from selenium.webdriver.chrome.service import Service 
from webdriver_manager.chrome import ChromeDriverManager 
from urllib.request import urlretrieve
from selenium.webdriver.common.by import By
from bs4 import BeautifulSoup

import pandas as pd
import datetime
import time

In [99]:
options = webdriver.ChromeOptions()
options.binary_location = r"C:\Users\kgw19\chromedriver-win64\chromedriver.exe" # Chrome의 경로를 지정해야 합니다.

driver = webdriver.Chrome(options=options)
coffeeBeanURL = "https://www.coffeebeankorea.com/store/store.asp"
driver.get(coffeeBeanURL)

# 커피빈 1번 매장에 대한 정보 크롤링

In [100]:
driver.execute_script("storePop2(31)") # 가장 첫번째로 보이는 매장 정보 크롤링

In [101]:
html = driver.page_source
soupCB1 = BeautifulSoup(html, "html.parser")

In [102]:
print(soupCB1.prettify())

<html class="js canvas canvastext no-touch geolocation postmessage indexeddb hashchange history draganddrop websockets rgba hsla multiplebgs backgroundsize borderradius boxshadow textshadow opacity cssanimations cssgradients cssreflections csstransforms csstransforms3d csstransitions fontface generatedcontent video audio localstorage sessionstorage webworkers no-applicationcache svg inlinesvg" lang="ko-KR" style="">
 <!--<![endif]-->
 <head>
  <script async="" src="https://www.googletagmanager.com/gtag/js?id=G-84W93E2CP2&amp;cx=c&amp;_slc=1" type="text/javascript">
  </script>
  <script async="" src="https://www.googletagmanager.com/gtag/js?id=G-EKXL62KETE&amp;cx=c&amp;_slc=1" type="text/javascript">
  </script>
  <script async="" src="//www.google-analytics.com/analytics.js">
  </script>
  <script async="" src="https://www.google-analytics.com/plugins/ua/linkid.js" type="text/javascript">
  </script>
  <meta charset="utf-8"/>
  <meta content="IE=edge, chrome=1" http-equiv="X-UA-Compat

In [103]:
# 매장이름 추출
store_name_h2 = soupCB1.select("div.store_txt > h2")
store_name_h2
store_name = store_name_h2[0].string
store_name

'삼성봉은사거리점'

In [104]:
# 매장주소 추출
store_address = soupCB1.select("div.store_txt > table > tbody:nth-child(1) > tr:nth-child(3) > td")
store_address

store_address_text = store_address[0].get_text(strip=True)
desired_info = store_address_text.split("<!--")[0]  # 주석 부분 제거
desired_info

'서울시 강남구 영동대로 607 1,2층'

In [105]:
# 매장전화번호 출력
store_mobile_number = soupCB1.select("div.store_txt > table > tbody:nth-child(1) > tr:nth-child(4) > td")
store_mobile_number_text = store_mobile_number[0].get_text(strip=True)
print(store_mobile_number_text)

02-3443-5618


In [110]:
# 흡연가능 여부(!!!!제일중요!!!!)
store_smoking_ok = soupCB1.select("div.store_txt > p > span.smoking")

smoking_element = next((span for span in store_smoking_ok if span.get("title") == "흡연가능"), None)
if smoking_element:
    yessmoking = smoking_element.get_text(strip=True)
yessmoking

'흡연가능'

In [63]:
sub_info = soupCB1.select("div.store_txt > table > tbody:nth-child(1) > tr:nth-child(5) > td")
sub_info_text = sub_info[0].get_text(strip=True)
print(sub_info_text)

식약처 인증 위생등급 매우 우수매장


# 매장 한 개에 대한 크롤링 작업을 매장의 갯수만큼 반복해서 처리

In [119]:
def CoffeeBean_store(result):
    for i in range(1, 30):
        driver.get(coffeeBeanURL)
        time.sleep(1)  # 웹페이지 연결할 동안 1초 대기
        try:
            driver.execute_script("storePop2(%d)" % i)
            time.sleep(1)  # 스크립트 실행할 동안 1초 대기
            html = driver.page_source
            soupCB1 = BeautifulSoup(html, "html.parser")
            # 매장이름 추출
            store_name_h2 = soupCB1.select("div.store_txt > h2")
            store_name = store_name_h2[0].string
            print(store_name)

            # 매장주소 추출
            store_address = soupCB1.select("div.store_txt > table > tbody:nth-child(1) > tr:nth-child(3) > td")
            store_address_text = store_address[0].get_text(strip=True)
            print(store_address_text)  # 주소 출력

            # 매장 전화번호 추출
            store_mobile_number = soupCB1.select("div.store_txt > table > tbody:nth-child(1) > tr:nth-child(4) > td")
            store_mobile_number_text = store_mobile_number[0].get_text(strip=True)
            print(store_mobile_number_text + "\n")

            # 흡연가능 여부
            store_smoking_ok = soupCB1.select("div.store_txt > p > span.smoking")
            smoking_element = next((span for span in store_smoking_ok if span.get("title") == "흡연가능"), None)
            if smoking_element:
                yes_smoke = smoking_element.get_text(strip=True)
            else:
                yes_smoke = "Nope"
                
            # '전화번호' 변수명 수정
            result.append([store_name] + [store_address] + [store_mobile_number_text] + [yes_smoke])
        except:
            continue

In [120]:
def main():
    result = []
    print("crawlling result")
    CoffeeBean_store(result)

    CB_table = pd.DataFrame(result, columns = ('매장', '주소', '전화번호', "흡연 가능 or 안됨"))
    CB_table.to_csv('./CoffeeBean.csv', encoding = 'cp949', mode = 'w', index = True)

In [123]:
if __name__ == "__main__":
    main()

crawlling result
차병원점
서울시 강남구 논현로 566 강남차병원1층
02-538-7615

강남대로점
서울시 서초구 강남대로 369 1층
02-588-5778

청담에스점
서울시 강남구 압구정로 461 네이처포엠빌딩B108,109호
02-548-6052

신사점
서울시 강남구 도산대로 126
02-548-2741

역삼점
서울시 강남구 논현로 512  지상1,2층
02-569-8051

양재스포타임점
서울시 서초구 강남대로 213 24호 지하1층
02-578-6833

청담성당점
서울시 강남구 삼성로 716 LEE76빌딩2층
02-542-2053

영동점
서울 서초구 반포동 736-17 P빌딩 2층
02-3443-2096

도곡점
서울시 강남구 언주로 30길 10,112 현대비젼21 112호
02-572-2781

영동고앞점
서울시 강남구 선릉로 749 1,2층
02-544-3794

압구정시티점
서울시 강남구 도산대로49길 13 1층 17,18호
02-543-5922

서초우성점
서울시 서초구 강남대로 51길 1 511 TOWER  1층
02-582-0158

논현팍스타워점
서울시 강남구 논현 231-13호 팍스타워지하1층
02-513-3870

삼성오크우드점
서울 강남구 테헤란로87길 46 지하 2층
02-3466-8507

